# Avance 2 — Ingeniería de Características (FE) + Selección/Extracción (CRISP-ML: Data Preparation)

**Objetivo:** predecir **Producción (kg)** y habilitar **optimización** del proceso.

**Entrega:** renombra este archivo como `Avance2.#Equipo.ipynb` antes de subirlo al repositorio del equipo.


In [ ]:
# ============================
# 0) Imports + Configuración
# ============================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

pd.set_option("display.max_columns", 200)

# Seaborn / statsmodels (opcionales)
try:
    import seaborn as sns
    HAVE_SNS = True
except Exception:
    HAVE_SNS = False

try:
    import statsmodels.api as sm
    HAVE_STATSMODELS = True
except Exception:
    HAVE_STATSMODELS = False

print("HAVE_SNS:", HAVE_SNS, "| HAVE_STATSMODELS:", HAVE_STATSMODELS)


## 1) Carga de datos (raw) y parsing de fechas

**Justificación:** convertir datos crudos del mundo real a un formato consistente y usable en ML; en particular, estandarizar timestamps para integrar clima por hora.


In [ ]:
# Rutas sandbox (este entorno) / rutas típicas de repo (data/)
SANDBOX_UNIFICADO = Path("/mnt/data/Dataset_Unificado.csv")
SANDBOX_CLIMA     = Path("/mnt/data/clima_horario_openmeteo_2025 (1).csv")

REPO_UNIFICADO = Path("../data/Dataset_Unificado.csv")
REPO_CLIMA     = Path("../data/clima_horario_openmeteo_2025.csv")

def read_csv_smart(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="ISO-8859-1")

if SANDBOX_UNIFICADO.exists() and SANDBOX_CLIMA.exists():
    df_raw = read_csv_smart(SANDBOX_UNIFICADO)
    clima_raw = read_csv_smart(SANDBOX_CLIMA)
    print("Leyendo desde /mnt/data (sandbox)")
else:
    df_raw = read_csv_smart(REPO_UNIFICADO)
    clima_raw = read_csv_smart(REPO_CLIMA)
    print("Leyendo desde ../data (repo)")

print("Dataset_Unificado:", df_raw.shape)
print("Clima:", clima_raw.shape)

df = df_raw.copy()
clima = clima_raw.copy()

df["ts_inicio"] = pd.to_datetime(df["Hora de inicio"], dayfirst=True, errors="coerce")
clima["ts_clima"] = pd.to_datetime(clima["time"], errors="coerce")

df = df.dropna(subset=["ts_inicio"]).sort_values("ts_inicio").reset_index(drop=True)
clima = clima.dropna(subset=["ts_clima"]).sort_values("ts_clima").reset_index(drop=True)

df.head()


## 2) Conversión de duraciones (texto → numérico)

**Justificación:** duraciones tipo `mm:ss` / `hh:mm` se convierten a segundos/minutos para permitir escalamiento, transformaciones y modelos.


In [ ]:
def mmss_to_seconds(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    if ":" not in s: return np.nan
    parts = s.split(":")
    if len(parts) != 2: return np.nan
    try:
        mm, ss = parts
        return int(mm) * 60 + int(ss)
    except ValueError:
        return np.nan

def hhmm_to_minutes(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    if ":" not in s: return np.nan
    parts = s.split(":")
    if len(parts) != 2: return np.nan
    try:
        hh, mm = parts
        return int(hh) * 60 + int(mm)
    except ValueError:
        return np.nan

if "Duración (mm:ss)" in df.columns:
    df["duracion_s"] = df["Duración (mm:ss)"].apply(mmss_to_seconds)
else:
    df["duracion_s"] = np.nan

if "Intervalo de ordeño (hh:mm)" in df.columns:
    df["intervalo_ordeño_min"] = df["Intervalo de ordeño (hh:mm)"].apply(hhmm_to_minutes)
else:
    df["intervalo_ordeño_min"] = np.nan

# Coerción a numérico de columnas típicas (si existen)
maybe_numeric = [
    "Producción (kg)",
    "Duracion de Incremento",
    "Duracion de Decremento",
    "Tiempo de Incremento",
    "Tiempo de Decremento",
    "Ubre",
    "Pezón",
    "Número de ordeño",
    "Num Lactacion",
]
for c in maybe_numeric:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df[[c for c in ["Duración (mm:ss)","duracion_s","Intervalo de ordeño (hh:mm)","intervalo_ordeño_min","Producción (kg)"] if c in df.columns]].head()


## 3) Integración con clima (enrichment)

**Justificación:** variables ambientales pueden afectar el proceso. Como el clima está por hora, se usa `merge_asof` con tolerancia ±2h.


In [ ]:
clima_feats = clima.rename(columns={
    "temperature_2m": "temp_C",
    "relative_humidity_2m": "hum_rel",
    "pressure_msl": "pres_msl",
    "precipitation": "precip_mm",
    "wind_speed_10m": "wind_ms",
}).copy()

clima_keep = ["ts_clima"]
for c in ["temp_C","hum_rel","pres_msl","precip_mm","wind_ms"]:
    if c in clima_feats.columns:
        clima_keep.append(c)
clima_feats = clima_feats[clima_keep].sort_values("ts_clima").reset_index(drop=True)

df = pd.merge_asof(
    df.sort_values("ts_inicio"),
    clima_feats,
    left_on="ts_inicio",
    right_on="ts_clima",
    direction="nearest",
    tolerance=pd.Timedelta("2H"),
)

df[["ts_inicio"] + [c for c in ["temp_C","hum_rel","pres_msl","precip_mm","wind_ms"] if c in df.columns]].head()


## 4) 2.3 — Generación de nuevas características

### 4.1 Temporales y cíclicas
**Justificación:** la periodicidad diaria/anual se representa con seno/coseno para evitar discontinuidades (23→0).

> Nota: se crea `is_weekend`, pero si resulta insignificante (ANOVA/correlación), se excluye del entrenamiento para reducir ruido.


In [ ]:
ts = df["ts_inicio"]
df["hour"] = ts.dt.hour
df["dow"] = ts.dt.dayofweek
df["month"] = ts.dt.month
df["dayofyear"] = ts.dt.dayofyear
df["is_weekend"] = (df["dow"] >= 5).astype(int)

df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24.0)
df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24.0)
df["doy_sin"]  = np.sin(2*np.pi*df["dayofyear"]/366.0)
df["doy_cos"]  = np.cos(2*np.pi*df["dayofyear"]/366.0)

df[["ts_inicio","hour","dow","month","dayofyear","is_weekend","hour_sin","hour_cos"]].head()


### 4.2 Eficiencia y dinámica del proceso
**Justificación:** para optimización interesa eficiencia (`kg/min`) y proporciones normalizadas (`inc_ratio`, `dec_ratio`).


In [ ]:
df["duracion_min"] = df["duracion_s"] / 60.0

if "Producción (kg)" in df.columns:
    df["kg_por_min"] = np.where(df["duracion_min"] > 0, df["Producción (kg)"] / df["duracion_min"], np.nan)
else:
    df["kg_por_min"] = np.nan

if "Tiempo de Incremento" in df.columns:
    df["inc_ratio"] = np.where(df["duracion_s"] > 0, df["Tiempo de Incremento"] / df["duracion_s"], np.nan)
if "Tiempo de Decremento" in df.columns:
    df["dec_ratio"] = np.where(df["duracion_s"] > 0, df["Tiempo de Decremento"] / df["duracion_s"], np.nan)

df[[c for c in ["Producción (kg)","duracion_min","kg_por_min","inc_ratio","dec_ratio"] if c in df.columns]].head()


## 5) Discretización / binning (2.3)

**Justificación:** capturar no-linealidad y producir insights operativos por rangos.


In [ ]:
if "duracion_min" in df.columns:
    df["duracion_bin_q"] = pd.qcut(df["duracion_min"], q=5, duplicates="drop")

if "temp_C" in df.columns:
    df["temp_bin"] = pd.cut(df["temp_C"], bins=[-10, 0, 10, 20, 30, 40, 60])

df[[c for c in ["duracion_min","duracion_bin_q","temp_C","temp_bin"] if c in df.columns]].head()


## 6) Codificación (2.3)

**Justificación:** ML requiere variables numéricas.  
- One-hot: baja cardinalidad  
- Frequency encoding: alta cardinalidad (evita explosión dimensional)


In [ ]:
cat_cols = [
    "Destino Leche",
    "Tipo de evento",
    "MS",
    "Usuario",
    "Usuario.1",
    "Programa de lavado",
    "Razón de la desviación",
    "duracion_bin_q",
    "temp_bin",
]
cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    df[c] = df[c].astype(str).str.strip()
    df.loc[df[c].isin(["nan","NaN","None",""]), c] = np.nan

def frequency_encode(series: pd.Series) -> pd.Series:
    freq = series.value_counts(dropna=True)
    return series.map(freq).fillna(0).astype(float)

high_card, low_card = [], []
for c in cat_cols:
    nuniq = df[c].nunique(dropna=True)
    (high_card if nuniq > 30 else low_card).append(c)

for c in high_card:
    df[c + "_freq"] = frequency_encode(df[c])

df_ohe = pd.get_dummies(df[low_card], prefix=low_card, dummy_na=True) if low_card else pd.DataFrame(index=df.index)

print("One-hot:", low_card)
print("Freq-encoding:", high_card)
df_ohe.head()


## 7) Matriz de features X y target y

Target: `Producción (kg)`


In [ ]:
target_col = "Producción (kg)"
assert target_col in df.columns, "No se encontró la columna target 'Producción (kg)'."

# Base numéricas + contexto + clima + ingeniería
base_feature_cols = [
    # Proceso
    "duracion_s","duracion_min","intervalo_ordeño_min",
    "Duracion de Incremento","Duracion de Decremento",
    "Tiempo de Incremento","Tiempo de Decremento",
    "Ubre","Pezón","Número de ordeño","Num Lactacion",
    # Tiempo (preferir cíclicas; is_weekend se excluye del set final por insignificancia frecuente)
    "hour","dow","month","dayofyear",
    "hour_sin","hour_cos","doy_sin","doy_cos",
    # Clima
    "temp_C","hum_rel","pres_msl","precip_mm","wind_ms",
    # Diagnóstico
    "kg_por_min","inc_ratio","dec_ratio",
]
base_feature_cols = [c for c in base_feature_cols if c in df.columns]

freq_cols = [c for c in df.columns if c.endswith("_freq")]

X_num = df[base_feature_cols + freq_cols].copy()
X_cat = df_ohe.copy()

X = pd.concat([X_num, X_cat], axis=1)
y = df[target_col].copy()

mask = y.notna()
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

print("X:", X.shape, "| y:", y.shape)
X.head()


## 7.1 Variables controlables vs no controlables (optimización)

**Justificación:** para optimizar, interesa entrenar/interpretar con variables que son *palancas* operativas.


In [ ]:
controlables = [
    "duracion_min",
    "duracion_s",
    "intervalo_ordeño_min",
    "Tiempo de Incremento",
    "Tiempo de Decremento",
    "Duracion de Incremento",
    "Duracion de Decremento",
]
controlables = [c for c in controlables if c in X.columns]
print("Controlables disponibles:", controlables)

X_control = X[controlables].copy()
print("X_control:", X_control.shape)


## 8) 2.4 — Mitigar sesgos / acelerar convergencia (imputación + escalamiento + transformaciones)

**Justificación:** muchos algoritmos no toleran NaNs y son sensibles a escala.  
Se implementa un bloque **robusto** para asegurar que las matrices entren a sklearn sin NaN y sin columnas vacías/constantes.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

def MakeSklearnSafe(Xtr: pd.DataFrame, Xte: pd.DataFrame):
    Xtr2 = Xtr.copy()
    Xte2 = Xte.copy()

    # 1) Eliminar columnas 100% NaN en train
    all_nan_cols = Xtr2.columns[Xtr2.isna().all()].tolist()
    if len(all_nan_cols) > 0:
        Xtr2 = Xtr2.drop(columns=all_nan_cols)
        Xte2 = Xte2.drop(columns=all_nan_cols)

    # 2) Imputar numéricas con mediana (mediana NaN -> 0), demás con 0
    num_cols = Xtr2.select_dtypes(include=[np.number]).columns.tolist()
    other_cols = [c for c in Xtr2.columns if c not in num_cols]

    med = Xtr2[num_cols].median(numeric_only=True).fillna(0)
    Xtr2[num_cols] = Xtr2[num_cols].fillna(med)
    Xte2[num_cols] = Xte2[num_cols].fillna(med)

    Xtr2[other_cols] = Xtr2[other_cols].fillna(0)
    Xte2[other_cols] = Xte2[other_cols].fillna(0)

    # 3) Quitar columnas constantes (no aportan para lineales/PCA/FA)
    stds = Xtr2[num_cols].std(numeric_only=True)
    const_cols = stds[stds < 1e-12].index.tolist()
    if len(const_cols) > 0:
        Xtr2 = Xtr2.drop(columns=const_cols)
        Xte2 = Xte2.drop(columns=const_cols)
        num_cols = [c for c in num_cols if c not in const_cols]

    # 4) Seguridad final
    Xtr2 = Xtr2.fillna(0)
    Xte2 = Xte2.fillna(0)

    return Xtr2, Xte2, num_cols

X_train_safe, X_test_safe, num_features_safe = MakeSklearnSafe(X_train, X_test)

# StandardScaler
scaler_std = StandardScaler()
X_train_std = X_train_safe.copy()
X_test_std  = X_test_safe.copy()
X_train_std[num_features_safe] = scaler_std.fit_transform(X_train_safe[num_features_safe])
X_test_std[num_features_safe]  = scaler_std.transform(X_test_safe[num_features_safe])

# MinMax (para chi2)
scaler_mm = MinMaxScaler()
X_train_mm = pd.DataFrame(scaler_mm.fit_transform(X_train_safe), columns=X_train_safe.columns, index=X_train_safe.index)
X_test_mm  = pd.DataFrame(scaler_mm.transform(X_test_safe), columns=X_test_safe.columns, index=X_test_safe.index)

# Yeo–Johnson (opcional; soporta ceros/negativos)
pt = PowerTransformer(method="yeo-johnson", standardize=True)
X_train_yj = X_train_safe.copy()
X_test_yj  = X_test_safe.copy()
if len(num_features_safe) > 0:
    X_train_yj[num_features_safe] = pt.fit_transform(X_train_safe[num_features_safe])
    X_test_yj[num_features_safe]  = pt.transform(X_test_safe[num_features_safe])

# Matrices controlables escaladas
controlables_std = [c for c in controlables if c in X_train_std.columns]
X_control_train_std = X_train_std[controlables_std].copy()
X_control_test_std  = X_test_std[controlables_std].copy()

print("X_train_std:", X_train_std.shape, "| X_train_mm:", X_train_mm.shape, "| X_train_yj:", X_train_yj.shape)
print("X_control_train_std:", X_control_train_std.shape)


## 9) Scatter plots (variables vs Producción) + curva por bins

**Justificación:** detectar no-linealidad, saturación y outliers; la curva por bins es interpretable y estable (sin dependencias extra).


In [ ]:
# Variables a graficar (ajusta a tu criterio)
vars_scatter = [
    "duracion_min",
    "intervalo_ordeño_min",
    "Tiempo de Incremento",
    "Tiempo de Decremento",
    "Duracion de Incremento",
    "Duracion de Decremento",
    "Num Lactacion",
    "Número de ordeño",
    "temp_C",
    "hum_rel",
    "precip_mm",
    "wind_ms",
    "kg_por_min",
    # 'is_weekend' suele ser insignificante; si quieres confirmarlo visualmente, descomenta:
    # "is_weekend",
]
vars_scatter = [v for v in vars_scatter if v in df.columns]

plot_df = df.loc[mask, vars_scatter + [target_col]].replace([np.inf, -np.inf], np.nan)

def binned_curve(x, y, bins=20):
    """Curva de respuesta por bins (media por bin) para interpretación operativa."""
    tmp = pd.DataFrame({"x": x, "y": y}).dropna()
    if len(tmp) < 50:
        return None
    tmp["bin"] = pd.cut(tmp["x"], bins=bins)
    g = tmp.groupby("bin", observed=True)["y"].mean()
    xmid = [iv.mid for iv in g.index]
    return np.array(xmid), g.values

for v in vars_scatter:
    x = plot_df[v]
    yv = plot_df[target_col]
    m = x.notna() & yv.notna()
    if m.sum() < 50:
        continue

    plt.figure(figsize=(6,4))
    plt.scatter(x[m], yv[m], alpha=0.25, s=12)
    out = binned_curve(x[m], yv[m], bins=25)
    if out is not None:
        xm, ym = out
        plt.plot(xm, ym, color="red", linewidth=2)  # tendencia en rojo
    plt.title(f"Producción vs {v} (scatter + bins)")
    plt.xlabel(v)
    plt.ylabel("Producción (kg)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


## 10) Selección de características (filtrado)

Incluye:
- Umbral de varianza
- Correlación (exploratoria)
- Chi-cuadrado (requiere clases -> discretizamos y)
- ANOVA (F-test) para regresión


In [ ]:
from sklearn.feature_selection import VarianceThreshold, chi2, f_regression

# 10.1 Umbral de varianza (sobre X_train_std)
vt = VarianceThreshold(threshold=0.01)
Xtr_vt = vt.fit_transform(X_train_std)
kept_cols_vt = X_train_std.columns[vt.get_support()].tolist()
print("VarThreshold: antes =", X_train_std.shape[1], "| después =", len(kept_cols_vt))

# 10.2 Correlación (sobre train, numéricas)
train_corr = pd.concat([X_train_safe.select_dtypes(include=[np.number]), y_train.reset_index(drop=True)], axis=1).corr(numeric_only=True)[target_col]
train_corr = train_corr.drop(labels=[target_col], errors="ignore").sort_values(ascending=False)
print("Top corr +:", train_corr.head(10).to_dict())
print("Top corr -:", train_corr.tail(10).to_dict())

# 10.3 Chi2 (para dependencia feature->clase). En regresión discretizamos y.
y_train_bins = pd.qcut(y_train, q=3, labels=["baja","media","alta"])
chi2_vals, p_vals = chi2(X_train_mm, y_train_bins)
chi2_df = pd.DataFrame({"feature": X_train_mm.columns, "chi2": chi2_vals, "p_value": p_vals}).sort_values("chi2", ascending=False)
chi2_df.head(15)


In [ ]:
# 10.4 ANOVA / F-test para regresión (requiere X sin NaN)
F, p = f_regression(X_train_std, y_train)
anova_df = pd.DataFrame({"feature": X_train_std.columns, "F": F, "p_value": p}).sort_values("F", ascending=False)
anova_df.head(15)


## 11) Extracción de características: PCA y Factor Analysis (FA)

**Justificación:** reducir dimensionalidad, multicolinealidad y costo computacional manteniendo información (PCA) o factores latentes (FA).


In [ ]:
from sklearn.decomposition import PCA, FactorAnalysis

# PCA (95% varianza) sobre X_train_std
pca = PCA(n_components=0.95, random_state=RANDOM_SEED)
Xtr_pca = pca.fit_transform(X_train_std)
Xte_pca = pca.transform(X_test_std)
print("Componentes PCA:", Xtr_pca.shape[1], "| Varianza acum:", float(pca.explained_variance_ratio_.sum()))


In [ ]:
# Factor Analysis: elegir un número razonable de factores (heurística)
n_factors = min(10, X_train_std.shape[1])
fa = FactorAnalysis(n_components=n_factors, random_state=RANDOM_SEED)
Xtr_fa = fa.fit_transform(X_train_std)
Xte_fa = fa.transform(X_test_std)
print("Factores FA:", Xtr_fa.shape[1])


## 12) Baseline de modelos (validación rápida)

**Justificación:** aunque el foco es Data Preparation, un baseline confirma señal y permite comparar representaciones.

Evaluamos:
- Ridge (std)
- Ridge (PCA)
- RandomForest (safe)
- Ridge (solo controlables) — **enfoque de optimización**


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

def eval_reg(model, Xtr, Xte, ytr, yte, name):
    model.fit(Xtr, ytr)
    pred = model.predict(Xte)
    mae = mean_absolute_error(yte, pred)
    r2 = r2_score(yte, pred)
    print(f"{name:>24} | MAE={mae:.4f} | R2={r2:.4f}")
    return mae, r2

ridge = Ridge(alpha=1.0, random_state=RANDOM_SEED)
eval_reg(ridge, X_train_std, X_test_std, y_train, y_test, "Ridge (std)")
eval_reg(ridge, Xtr_pca, Xte_pca, y_train, y_test, "Ridge (PCA)")

rf = RandomForestRegressor(n_estimators=400, random_state=RANDOM_SEED, n_jobs=-1)
eval_reg(rf, X_train_safe, X_test_safe, y_train, y_test, "RF (safe)")

# Solo controlables (más interpretable para optimización)
if X_control_train_std.shape[1] >= 1:
    eval_reg(ridge, X_control_train_std, X_control_test_std, y_train, y_test, "Ridge (controlables)")
else:
    print("No hay suficientes variables controlables disponibles para entrenar Ridge(controlables).")


## 13) Conclusiones — CRISP-ML (Data Preparation)

1) Se estandarizaron timestamps y se integró clima por hora (enrichment).  
2) Se generaron features de eficiencia/dinámica (kg/min, ratios) y temporales cíclicas.  
3) Se aplicó binning para insights operativos.  
4) Se codificaron categóricas con one-hot y frequency encoding.  
5) Se implementó un preprocesamiento robusto (imputación + eliminación de columnas vacías/constantes + escalamiento).  
6) Se aplicaron filtros (varianza, correlación, chi2, ANOVA) y extracción (PCA, FA).  

**Hallazgo de relevancia:** variables de calendario como `is_weekend` suelen resultar **insignificantes** en este dominio (operación estandarizada). Por ello se excluyó `is_weekend` del set de entrenamiento final para reducir ruido y dimensionalidad, manteniendo representaciones cíclicas (`hour_sin/cos`) más informativas.

**Siguiente paso (Modeling/Optimization):**
- Interpretabilidad enfocada en **variables controlables** (palancas) y búsqueda de “sweet spots” (p.ej., duración óptima) con restricciones operativas.


## 14) Export (opcional)

Exporta el dataset engineered para usar en la siguiente fase.


In [ ]:
engineered = X.copy()
engineered[target_col] = y

OUT = Path("/mnt/data/engineered_produccion_avance2.csv")
engineered.to_csv(OUT, index=False)
print("Exportado:", OUT, "| shape:", engineered.shape)
engineered.head()


---
### Checklist antes de entregar
1. Kernel → Restart & Run All  
2. Verifica que corre de principio a fin (secuencial).  
3. Renombra a `Avance2.#Equipo.ipynb` con tu número real.  
